# Metadata Silver

**Capa:** Silver

---


### Step 1
**Objetivo:** Define la estructura de las tablas limpias en la capa Silver.

**Resumen Tablas:**
- airbnb.silver.listings;
- airbnb.silver.hosts;
- airbnb.silver.reviews;
- airbnb.silver.amenities;
- airbnb.silver.listing_amenities;
- airbnb.silver.address;

---


In [0]:
-- Elimina tablas si existen
DROP TABLE IF EXISTS airbnb.silver.listings;
DROP TABLE IF EXISTS airbnb.silver.hosts;
DROP TABLE IF EXISTS airbnb.silver.reviews;
DROP TABLE IF EXISTS airbnb.silver.amenities;
DROP TABLE IF EXISTS airbnb.silver.listing_amenities;
DROP TABLE IF EXISTS airbnb.silver.address;

-- -------------------------------------------
-- SILVER.LISTINGS
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.listings (
  listing_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  bk_listing_id STRING NOT NULL,

  name STRING,
  summary STRING,
  description STRING,
  property_type STRING,
  room_type STRING,
  accommodates INT,
  bedrooms DOUBLE,
  beds DOUBLE,
  bathrooms DOUBLE,
  price DECIMAL(12,2),
  minimum_nights INT,
  maximum_nights INT,
  instant_bookable BOOLEAN,

  -- host referencia (BK + SK para resolver después)
  bk_host_id STRING,
  host_sk BIGINT,

  first_review_date DATE,
  last_review_date DATE,
  number_of_reviews INT,
  review_scores_rating DOUBLE,

  -- ubicación básica (normalizada en address)
  latitude DOUBLE,
  longitude DOUBLE,
  neighborhood STRING,
  city STRING,
  country STRING,

  ingest_ts TIMESTAMP
)
USING DELTA
PARTITIONED BY (country)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true',
  'delta.enableChangeDataFeed'='true'
);

-- -------------------------------------------
-- SILVER.HOSTS
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.hosts (
  host_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  bk_host_id STRING NOT NULL,

  host_name STRING,
  host_response_time STRING,
  host_response_rate INT,
  host_is_superhost BOOLEAN,
  host_total_listings_count INT,
  host_identity_verified BOOLEAN,
  host_location STRING,

  ingest_ts TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true'
);

-- -------------------------------------------
-- SILVER.REVIEWS
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.reviews (
  review_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  bk_review_id STRING,

  listing_sk BIGINT,
  bk_listing_id STRING,

  reviewer_id STRING,
  reviewer_name STRING,
  review_date DATE,
  comments STRING,
  comments_len INT,

  ingest_ts TIMESTAMP
)
USING DELTA
PARTITIONED BY (review_date)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true'
);

-- -------------------------------------------
-- SILVER.AMENITIES
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.amenities (
  amenity_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  bk_amenity_name STRING NOT NULL,

  active BOOLEAN,
  ingest_ts TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true'
);

-- -------------------------------------------
-- SILVER.LISTING_AMENITIES
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.listing_amenities (
  listing_sk BIGINT,
  amenity_sk BIGINT,

  bk_listing_id STRING,
  bk_amenity_name STRING,
  source_tag STRING
)
USING DELTA
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true'
);

-- -------------------------------------------
-- SILVER.ADDRESS
-- -------------------------------------------
CREATE OR REPLACE TABLE airbnb.silver.address (
  address_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  listing_sk BIGINT,
  bk_listing_id STRING,

  street STRING,
  neighborhood STRING,
  city STRING,
  state STRING,
  country STRING,
  country_code STRING,
  latitude DOUBLE,
  longitude DOUBLE,

  ingest_ts TIMESTAMP
)
USING DELTA
PARTITIONED BY (country)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite'='true',
  'delta.autoOptimize.autoCompact'='true'
);